In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from transformers import AutoProcessor, AutoModelForImageClassification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/users/PAS2136/rayees/CV4A - Efficient Probing/ProbingSubsets/probing_full_dataset.csv"
df = pd.read_csv(file_path)
cols = ['ImageFilePath', 'ScientificName']
df = df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Asaphidion curtum
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Asaphidion curtum


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "microsoft/swinv2-base-patch4-window8-256"

processor = AutoProcessor.from_pretrained(model_name, use_fast=True)

model = AutoModelForImageClassification.from_pretrained(model_name).to(device).eval()


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.logits
    
    return features.cpu().numpy()


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing Images ...") ])

seed = 99

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X, y, df_indices, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)


Processing Images ...:   0%|          | 0/63077 [00:00<?, ?it/s]

In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

metrics = {}

predictions = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")

metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 12.55% | Prec: 21.62% | Rec: 12.55% | F1: 12.38% | Bal Acc: 16.44% | MCC: 0.1237
LogisticRegression        | Acc: 78.03% | Prec: 78.32% | Rec: 78.03% | F1: 77.92% | Bal Acc: 75.63% | MCC: 0.7791
NearestNeighbor           | Acc: 34.17% | Prec: 37.86% | Rec: 34.17% | F1: 34.11% | Bal Acc: 29.81% | MCC: 0.3382
MLP_Baseline              | Acc: 77.62% | Prec: 77.89% | Rec: 77.62% | F1: 77.54% | Bal Acc: 75.81% | MCC: 0.7750


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

In [10]:
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
35206,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Nebria brevicollis,Calathus rotundicollis,Nebria salina,Leistus spinibarbis,Nebria salina
17377,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Britis...,Notiophilus rufipes,Dromius angustus,Notiophilus biguttatus,Agonum thoreyi,Notiophilus rufipes


In [11]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.125476,0.21622,0.125476,0.123813,0.164422,0.1237
LogisticRegression,LogisticRegression,0.780279,0.78315,0.780279,0.779169,0.756319,0.779134
NearestNeighbor,NearestNeighbor,0.341709,0.378634,0.341709,0.341106,0.2981,0.338203
MLP_Baseline,MLP_Baseline,0.776157,0.77885,0.776157,0.775406,0.75813,0.775008


In [12]:

basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_full/'


In [13]:

test_df.to_csv(f"{basedir}/5.SWINv2-species.csv", index=False)

metrics_df.to_csv(f"{basedir}/5.SWINv2-species-metrics.csv", index=False)

